# Geoapify Places 5 Cities

In [32]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np

# Import the API key
from config_key import geoapify_key

In [33]:
#Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
cities = ["Bronx, NY","Napa, CA", "Victorville, CA","San Francisco, CA","Monmouth, OR"]
latitude = [40.837048,38.297539,34.536217,37.828724,44.8485]
longitude = [-73.865433,-122.286865,-117.292763,-122.355537,-123.2340]

#Create DataFrame called US_locations_df to store the city, coordinates, supermarket property, distance
US_locations_df = pd.DataFrame({
                            "cities": cities, 
                            "latitude": latitude, 
                            "longitude": longitude})

#Add empty columns to the DataFrame to store, supermarket (property name), distance, etc. 
US_locations_df["supermarket"] = ""
US_locations_df["distance"] = ""

US_locations_df

,cities,latitude,longitude,supermarket,distance
0,"Bronx, NY",40.837048,-73.865433,,
1,"Napa, CA",38.297539,-122.286865,,
2,"Victorville, CA",34.536217,-117.292763,,
3,"San Francisco, CA",37.828724,-122.355537,,
4,"Monmouth, OR",44.848500,-123.234000,,


In [46]:
# Set the parameters to search for supermarkets 
categories = "commercial.supermarket"
radius = 16200

# Set up a params dictionary
params = {
    "categories":categories,
    "apiKey":geoapify_key,
    "limit": 100
}

# Print a message to follow up the supermarket search
print("Starting supermarket search...") 

# Iterate through the US_locations_df DataFrame
for index, row in US_locations_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["longitude"]
    latitude = row["latitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    response = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        US_locations_df.loc[index, "supermarket"] = response["features"][0]["properties"]["address_line1"]
        US_locations_df.loc[index, "distance"] = response["features"][0]["properties"]["distance"] 
        
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        US_locations_df.loc[index, "supermarket"] = "No supermarket found"
        US_locations_df.loc[index, "distance"] = "No distance found"
      
    # Log the search results
    print(f"{US_locations_df.loc[index, 'cities']} - nearest supermarket: {US_locations_df.loc[index, 'supermarket']}")

# Display sample data
US_locations_df


Starting supermarket search...
Bronx, NY - nearest supermarket: Bravo
Napa, CA - nearest supermarket: Walmart Supercenter
Victorville, CA - nearest supermarket: Food 4 Less
San Francisco, CA - nearest supermarket: Island Cove Market
Monmouth, OR - nearest supermarket: Western Oregon University Book Store


,cities,latitude,longitude,supermarket,distance
0,"Bronx, NY",40.837048,-73.865433,Bravo,138
1,"Napa, CA",38.297539,-122.286865,Walmart Supercenter,1254
2,"Victorville, CA",34.536217,-117.292763,Food 4 Less,2734
3,"San Francisco, CA",37.828724,-122.355537,Island Cove Market,1338
4,"Monmouth, OR",44.848500,-123.234000,Western Oregon University Book Store,519
